In [2]:
import os,sys
import geopandas as gpd
from functions import *
from shapely.ops import cascaded_union
from rasterstats import zonal_stats
import numpy as np
import fiona
from shapely.geometry import mapping,MultiPolygon, LineString
import pandas as pd
import logging
import requests,zipfile,io
import urllib

# Start the logging processus
logging.basicConfig(filename='log_objectif.log',level=logging.INFO,\
format='%(asctime)s,%(msecs)d %(levelname)-8s [%(filename)s:%(lineno)d] %(message)s')

## Initialization

In [3]:
#%% Setting Parameters
country = 'TUN'
# Define Base Path
base_path = os.getcwd()

# Read the List of Countries having a Level of Completeness >0.75 and Good Ratio RAI PST/RAI PS
df_countries = pd.read_excel(os.path.join(base_path,'Input','list of countries.xlsx'))

# Read useful informations about developing countries (Region, Kms of Roads, Cost of Upgrading tertiary Roads...)
df_roads_tot = pd.read_excel(os.path.join(base_path,'Input','Roads_for_code.xlsx'))

# Continent of the country
continent_osm = df_roads_tot.set_index('ISO3').loc[country,'Continent']

country_full = df_roads_tot.set_index('ISO3').loc[country,'Country']
country_full = country_full.lower().replace(' ','-')

if country_full=='kyrgyz-republic':
    country_full = 'kyrgyzstan'

output_dir = os.path.join(base_path,'output')

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
output_country = os.path.join(output_dir,country)

if not os.path.exists(output_country):
    os.makedirs(output_country)

### Country Boundaries

In [4]:
logging.info('%s has started' %country)
# set path for global country shapes
world_shp_path = os.path.join('input_data','World_boundaries')
if not os.path.exists(world_shp_path):
    os.makedirs(world_shp_path)
    
    # Downloading and extracting file
    url_world_shp = 'https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/50m/cultural/ne_50m_admin_0_countries.zip'
    r = requests.get(url_world_shp, stream=True)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(world_shp_path)
    
world_shp = gpd.read_file(os.path.join(world_shp_path,'ne_50m_admin_0_countries.shp'))
country_boundary = world_shp[world_shp.ADM0_A3==country.upper()]

# Storing the country boundary geometry
geo_country = country_boundary.unary_union

### Urban Areas

In [5]:
# Set the path for the Grump Database
# To download from https://sedac.ciesin.columbia.edu/data/set/grump-v1-urban-ext-polygons-rev01/data-download
grump = os.path.join(base_path,'input_data','GRUMP','global_urban_extent_polygons_v1.01.shp')

# Reading the urban shapefile
urban_geoms = gpd.read_file(grump)
# Defining the crs of the shapefile
urban_geoms.crs = {'init' :'epsg:4326'}
# Selecting the urban areas of the country
urban_geoms = urban_geoms.loc[urban_geoms['ISO3']==country]

urban_geoms_exact = urban_geoms.copy()

### World Pop

In [6]:
worldpop_path = os.path.join(base_path,'input_data','Worldpop')

if not os.path.exists(worldpop_path):
    os.makedirs(worldpop_path)

#set global file paths for worldpop
if ('central-america' in continent_osm) or ('south-america' in continent_osm) :
    # Downloading population file
    world_pop = os.path.join(base_path,'input_data','Worldpop','LAC_PPP_2015_adj_v2.tif') 
    if not os.path.exists(world_pop):
        url_worldpop = 'ftp://ftp.worldpop.org.uk/GIS/Population/Whole_Continent/Latin_America_and_the_Caribbean_1km_Population/LAC_PPP_2020_adj_v2.tif'
        urllib.request.urlretrieve(url_worldpop, world_pop)

    
elif ('africa' in continent_osm):
    # Downloading population file
    world_pop = os.path.join(base_path,'input_data','Worldpop','AFR_PPP_2015_adj_v2.tif')
    if not os.path.exists(world_pop):
        url_worldpop = 'ftp://ftp.worldpop.org.uk/GIS/Population/Whole_Continent/Africa_1km_Population/AFR_PPP_2020_adj_v2.tif'
        urllib.request.urlretrieve(url_worldpop, world_pop)
    
elif ('asia' in continent_osm):
    # Downloading population file
    world_pop = os.path.join(base_path,'input_data','Worldpop','Asia_PPP_2015_adj_v2.tif') 
    if not os.path.exists(world_pop):
        url_worldpop = 'ftp://ftp.worldpop.org.uk/GIS/Population/Whole_Continent/Asia_1km_Population/Asia_PPP_2020_adj_v2.tif'
        urllib.request.urlretrieve(url_worldpop, world_pop)

elif ('europe' in continent_osm):
    world_pop = os.path.join(base_path,'Worldpop','EUROPOP_WGS84.tif') 
    
if country == 'MEX':
    world_pop = os.path.join(base_path,'Worldpop','MEX_ppp_v2c_2015_UNadj.tif')   
elif country == 'UKR':
    world_pop = os.path.join(base_path,'Worldpop','UKR_ppp_v2b_2015_UNadj.tif')

### Country Roads

In [7]:
country_roads_path = os.path.join('input_data','Country_data')
specific_country_roads_path = os.path.join(country_roads_path,country)
roads_shp_path = os.path.join(specific_country_roads_path,'gis_osm_roads_free_1.shp')

if not os.path.exists(roads_shp_path):
    os.makedirs(specific_country_roads_path)
    # Downloading and extracting file
    country_roads_shp = 'https://download.geofabrik.de/%s/%s-latest-free.shp.zip'%(continent_osm,country_full)
    try:
        r = requests.get(country_roads_shp, stream=True)
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall(specific_country_roads_path)
    except:
        country_roads_shp = 'https://download.geofabrik.de/central-america/%s-latest-free.shp.zip'%(country_full)
        r = requests.get(country_roads_shp, stream=True)
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall(specific_country_roads_path)
    
for dir in os.listdir(specific_country_roads_path):
    if 'roads' not in dir:
        os.remove(os.path.join(specific_country_roads_path,dir))
        
load_country = gpd.read_file(roads_shp_path)

uniq = load_country['fclass'].value_counts()
uniq = list(uniq[uniq > 20].index)
uniq.extend(['primary','secondary','trunk','motorway'])
uniq = list(set(uniq))
load_country = load_country[load_country['fclass'].isin(uniq)]

load_country_tot = map_roads(load_country)

In [8]:
#%% Extracting shape of the country,rural buffer and computing the total population living 
# in rural areas

# Remove urban geometries outside of country boundaries
urban_within_country = gpd.sjoin(urban_geoms_exact,country_boundary,op='within')
urban_geoms_exact['within_country'] = False
urban_geoms_exact.loc[urban_within_country.index, 'within_country'] = True

urban_geoms_exact = urban_geoms_exact[urban_geoms_exact.is_valid==True]
urban_geoms_exact['geometry'] = urban_geoms_exact.apply(lambda x : geom_within_country(x,geo_country),axis=1)
urban_geoms_exact['geometry'] = urban_geoms_exact['geometry'].buffer(0)

# Extract rural areas by doing a difference between country boundary and clipped urban geometries
rural_buffer = country_boundary.unary_union.difference(urban_geoms_exact.unary_union)
        
# Compute the centroid of the country
country_centroid = country_boundary.centroid.values[0]
# Getting the longitude and latitude of the country
lat,lon = country_centroid.bounds[1],country_centroid.bounds[0]
# Compute the epsg of a new projection to create buffers around roads
epsg = int(32700-np.round((45+lat)/90,0)*100+np.round((183+lon)/6,0))
# Compute the rural population of the country
stats_rur = sum(item['sum'] for item in zonal_stats(rural_buffer,world_pop,
                    stats="sum") if item['sum'] is not None)
        
logging.info('End of the Extraction of country boundary,rural buffer and rural population')

C:\Users\mehdi\anaconda3\envs\GDAL\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!(+init=epsg:4326 +type=crs != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)


In [9]:
#%% Loading all the roads for the country
logging.info('Loading Roads')
load_country_tot = load_country_tot.loc[load_country_tot['roads'].isin(['primary','secondary','tertiary'])]
logging.info('End of loading Roads')

# Getting rid of roads outside of country boundary

roads_within_country = gpd.sjoin(load_country_tot,country_boundary,op='within')
load_country_tot['within_country'] = False
load_country_tot.loc[roads_within_country.index, 'within_country'] = True
load_country_tot['geometry'] = load_country_tot.apply(lambda x: geom_within_country(x,geo_country),axis=1)
load_country_tot = load_country_tot[~load_country_tot.geometry.is_empty]
# Compute new roads' length
load_country_tot['distance'] = load_country_tot['geometry'].apply(line_length)


C:\Users\mehdi\anaconda3\envs\GDAL\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\mehdi\anaconda3\envs\GDAL\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\mehdi\anaconda3\envs\GDAL\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [10]:
#%% Reduce Roads and remove roads in cities

# Extract tertiary roads 
load_country_ter = load_country_tot.loc[load_country_tot['roads'].isin(['tertiary'])]
# Renaming the column distance
load_country_ter = load_country_ter.rename(index=str, columns={"distance":"Length_Road"})
# Remove roads shorter than 500 meters
load_country_ter = load_country_ter.loc[load_country_ter.Length_Road>=0.5]

geo_urban = urban_geoms_exact.unary_union
ter_roads_inter_urban = gpd.sjoin(load_country_ter,urban_geoms_exact,op='intersects')
load_country_ter['inter_urb'] = False
load_country_ter.loc[ter_roads_inter_urban.index, 'inter_urb'] = True

load_country_ter['geometry'] = load_country_ter.apply(lambda x: delete_roads_urb(x,geo_urban),axis=1)
load_country_ter = load_country_ter[~load_country_ter.geometry.is_empty]

# Computing roads length
ter_rural = load_country_ter.copy()
ter_rural['Length_Roads'] = ter_rural['geometry'].to_crs(epsg=epsg).length/1000
ter_rural = ter_rural[ter_rural.Length_Roads>=0.5]

C:\Users\mehdi\anaconda3\envs\GDAL\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!(epsg:4326 != +init=epsg:4326 +type=crs)
  "(%s != %s)" % (left_df.crs, right_df.crs)


In [11]:
logging.info('Beginning of the step of reducing roads')

# Create a 200m Buffer around tertiary roads
buffer_ter_200 = Create_Buffer(ter_rural,epsg,200) 

In [12]:
#%%Create the Connected Rural Buffer PS 
logging.info('Starting the creation of the rural buffer')
# Select Primary and Secondary Roads from the total roads
load_country_PS = load_country_tot.loc[load_country_tot['roads'].isin(['primary','secondary'])]
# Copy the network of Primary & Secondary Roads
load_country_PS_buf = load_country_PS.copy()
# Create a 200 meters Buffer around PS roads
load_country_PS_buf = Create_Buffer(load_country_PS_buf,epsg,200)

# Dissolve roads and create new GeoDataFrame
PS_roads_geo = load_country_PS_buf.unary_union
try:
    buffer_PS = pd.DataFrame({'geometry':PS_roads_geo})
    buffer_PS = gpd.GeoDataFrame(buffer_PS, crs = {'init' :'epsg:4326'}, geometry = 'geometry')
except:
    buffer_PS = pd.DataFrame({'geometry':PS_roads_geo},index={0})
    buffer_PS = gpd.GeoDataFrame(buffer_PS, crs = {'init' :'epsg:4326'}, geometry = 'geometry')
    
# Change the crs of the shapefile
buffer_PS = buffer_PS.to_crs(epsg=epsg)
# Compute the area of each polygon
buffer_PS['area'] = buffer_PS['geometry'].area
# Rechange the crs to the WGS84 projection
buffer_PS = buffer_PS.to_crs(epsg=4326)
# Rank roads by area (descending)
buffer_PS = buffer_PS.sort_values(by=['area'], ascending=False)
# Copy the Buffer PS
buffer_PS1 = buffer_PS.copy()
# Add the 3 roads having the biggest geometry
buffer_PS = buffer_PS[0:3]
# Add all roads having an area bigger than 60 km² (with a 200m buffer it's basically roads having a length bigger than 150km)
try:
    buffer_PS = buffer_PS.append(buffer_PS1.loc[buffer_PS1['area']>=60000000])
    buffer_PS = buffer_PS[~buffer_PS.index.duplicated(keep='first')].reset_index(drop=True)
except:
    buffer_PS = buffer_PS.reset_index(drop=True)
logging.info('End of the creation of the rural buffer')

In [13]:
#%%Check if the tertiary roads intersect the rural PS buffer
logging.info('Checking which tertiary roads are directly connected to the network of Primary and Secondary Roads ')
# Select the geometry of the connected PS network
poly_PS_network = buffer_PS.geometry.unary_union
# Checking if Tertiary Roads are connected to the connected network of PS roads
buffer_ter_200['Linked_Network'] = buffer_ter_200.geometry.intersects(poly_PS_network)

logging.info('End of the intersection of Tertiary Roads with Rural Buffer')


In [14]:
#%% Properly Indexing Roads
buffer_ter_200 = buffer_ter_200.reset_index(drop=True)
buffer_ter_200['Col_index'] = buffer_ter_200.index

buffer_ter_200['Num_road'] = buffer_ter_200.Col_index.apply(lambda x :'r_'+str(x))
#%%Intersect the Buffer of tertiary roads with Rural areas
logging.info('Intersecting 2000 Buffer Tertiary Roads with Rural Buffer')
# Create a 1800m Buffer from the initial 200m
ter_roads_2000 = Create_Buffer(buffer_ter_200,epsg,1800)
ter_roads_2000['geometry'] = ter_roads_2000.geometry.buffer(0)

# Removing urban areas from buffered 2km tertiary roads
ter_roads_2000['inter_urb'] = ter_roads_2000.geometry.intersects(geo_urban.buffer(0))
ter_roads_2000['geometry'] = ter_roads_2000.apply(lambda x: delete_roads_urb(x,geo_urban),axis=1)

# Removing empty geometries
ter_roads_2000 = ter_roads_2000[~ter_roads_2000.geometry.is_empty]

# Removing from the 200m Tertiary Roads urban roads that have disappeared from the clip
buffer_ter_200 = buffer_ter_200.loc[buffer_ter_200['Num_road'].isin(ter_roads_2000['Num_road'])]
logging.info('End of intersecting 2000 Buffer Tertiary Roads with Rural Buffer')



In [15]:
#%%Intersecting the Buffer PS 2000 with rural areas
logging.info('Intersecting 2000 Buffer Roads with Rural Buffer')
# Create the 2km Buffer by adding 1800m to the previous 200m
load_country_PS_buf = Create_Buffer(buffer_PS, epsg, 1800)
load_country_PS_buf['geometry'] = load_country_PS_buf.geometry.buffer(0)

# Removing urban areas from buffered 2km tertiary roads
load_country_PS_buf['inter_urb'] = load_country_PS_buf.geometry.intersects(geo_urban)
load_country_PS_buf['geometry'] = load_country_PS_buf.apply(lambda x: delete_roads_urb(x,geo_urban),axis=1)
# Removing empty geometries
load_country_PS_buf = load_country_PS_buf[~load_country_PS_buf.geometry.is_empty]

logging.info('End of intersecting 2000 Buffer Roads with Rural Buffer')
#%%Compute the number of Rural Inhabitants living next to PS Roads
Pop_rur_PS = sum(item['sum'] for item in zonal_stats(load_country_PS_buf,world_pop,
                        stats="sum") if item['sum'] is not None)

In [16]:
logging.info('Computing the population around tertiary roads')
Population_ter_roads = zonal_stats(ter_roads_2000,world_pop,stats="sum")
Population_ter_roads = [item['sum'] for item in Population_ter_roads]
ter_roads_2000['Population'] = Population_ter_roads

In [17]:
# Removing population from tertiary roads intersecting PS Network
inter_PS_ter = ter_roads_2000[ter_roads_2000.Linked_Network==True].intersection(load_country_PS_buf.unary_union)
Pop_inter = zonal_stats(inter_PS_ter,world_pop,stats="sum")
Pop_inter = np.array([item['sum'] for item in Pop_inter])
Pop_inter[Pop_inter==None] = 0

ter_roads_2000['New_People'] = ter_roads_2000['Population']
ter_roads_2000.loc[ter_roads_2000[ter_roads_2000.Linked_Network==True].index,'New_People']  = \
ter_roads_2000.loc[ter_roads_2000[ter_roads_2000.Linked_Network==True].index,'Population']  - Pop_inter


ter_roads_2000 = ter_roads_2000.set_index('Num_road')
buffer_ter_200 = buffer_ter_200.set_index('Num_road')
buffer_ter_200['New_People'] = ter_roads_2000['New_People']

In [18]:
#%%Method to compute the successive RAIs. At each iteration, we take the road that will increase the most the RAI and we 
#add it to the network  

# Getting the cost of upgrading tertiary roads to paved roads in the country
cost_per_km = df_roads_tot.set_index('ISO3').loc[country,'Cost Upgrade']/1000000

# ter_roads_2000 = ter_roads_2000.set_index('Num_road')
# buffer_ter_200 = buffer_ter_200.set_index('Num_road')
buffer_ter_200['New_People'] = ter_roads_2000['New_People']

# Create output GeoDataFrame for tertiary roads buffer 2km
df = gpd.GeoDataFrame()
# Create output GeoDataFrame for tertiary roads buffer 200m
df_200 = gpd.GeoDataFrame()

# Removing roads shorter than 500m
ter_roads_2000 = ter_roads_2000[ter_roads_2000.Length_Roads>=0.5]
buffer_ter_200 = buffer_ter_200[buffer_ter_200.Length_Roads>=0.5]

# Removing roads with a population lower than Pop_rur_PS/10000
ter_roads_2000 = ter_roads_2000[ter_roads_2000.New_People>=Pop_rur_PS/10000]
buffer_ter_200 = buffer_ter_200[buffer_ter_200.New_People>=Pop_rur_PS/10000]

ter_roads_2000['People_per_km'] = (ter_roads_2000.New_People/(ter_roads_2000.Length_Roads))
buffer_ter_200['People_per_km'] = buffer_ter_200.New_People/(buffer_ter_200.Length_Roads)

# Ranking roads from the biggest People per km to the lowest
ter_roads_2000 = ter_roads_2000.sort_values(by=['People_per_km'], ascending=False)
buffer_ter_200 = buffer_ter_200.sort_values(by=['People_per_km'], ascending=False)

# # Select the network of connected roads
# roads_2km = ter_roads_2000.copy()
# roads_200m = buffer_ter_200.copy()

# Selecting the network of non connected roads
non_connected_roads_2000 = ter_roads_2000[ter_roads_2000.Linked_Network==False]
non_connected_roads_200 = buffer_ter_200[buffer_ter_200.Linked_Network==False].copy()

# Select the network of connected roads
connected_roads_2000 = ter_roads_2000[ter_roads_2000.Linked_Network==True]
connected_roads_200 = buffer_ter_200[buffer_ter_200.Linked_Network==True]

# Add first road to output
df = df.append(connected_roads_2000[0:1]).reset_index(drop=True)
df_200 = df_200.append(connected_roads_200[0:1]).reset_index(drop=True)

# Remove the first road added from the dafaframe of tertiary to be added
connected_roads_2000 = connected_roads_2000[1:]
connected_roads_200 = connected_roads_200[1:]

# Adding new connected roads (from non connected roads) to the pool of connected roads
non_connected_roads_200['inter_new_geo'] = non_connected_roads_200.intersects(df_200.unary_union)
index_to_add = non_connected_roads_200[non_connected_roads_200['inter_new_geo']==True].index

connected_roads_200 = connected_roads_200.append(non_connected_roads_200.loc[index_to_add,:])
non_connected_roads_200 = non_connected_roads_200.drop(index_to_add)

connected_roads_2000 = connected_roads_2000.append(non_connected_roads_2000.loc[index_to_add,:])
non_connected_roads_2000 = non_connected_roads_2000.drop(index_to_add)

In [19]:
i=0
while (len(connected_roads_2000)!=0):
    connected_roads_2000['inter_new_geo'] = connected_roads_2000.intersects(df.loc[i,'geometry'])
    inter_new_con = connected_roads_2000[connected_roads_2000.inter_new_geo==True].intersection(df.loc[i,'geometry'])   
    
    if len(inter_new_con)>0:
        Pop_inter = zonal_stats(inter_new_con,world_pop,stats="sum")
        Pop_inter = np.array([item['sum'] for item in Pop_inter])
        Pop_inter[Pop_inter==None] = 0
        index_inter = connected_roads_2000[connected_roads_2000.inter_new_geo==True].index
        connected_roads_2000.loc[index_inter,'New_People'] = connected_roads_2000.loc[index_inter,'New_People'] - Pop_inter


    # Updating population
    connected_roads_200['New_People'] = connected_roads_2000['New_People']
    
    # Reducing number of roads
    connected_roads_2000 = connected_roads_2000[connected_roads_2000.New_People>=Pop_rur_PS/10000]
    connected_roads_200 = connected_roads_200[connected_roads_200.New_People>=Pop_rur_PS/10000]
    
    # Updating population per km
    connected_roads_2000['People_per_km'] = (connected_roads_2000.New_People/connected_roads_2000.Length_Roads)                                                                     
    connected_roads_200['People_per_km'] = connected_roads_2000['People_per_km']
    
    # Ranking roads from the biggest People per km to the lowest
    connected_roads_2000 = connected_roads_2000.sort_values(by=['People_per_km'], ascending=False)
    connected_roads_200 = connected_roads_200.sort_values(by=['People_per_km'], ascending=False)

    # Adding the new selected road
    df = df.append(connected_roads_2000[0:1]).reset_index(drop=True)
    df_200 = df_200.append(connected_roads_200[0:1]).reset_index(drop=True)

    # Keeping the last road added
    df_po = connected_roads_200[0:1]

    #Removing the last road added from the network
    connected_roads_2000 = connected_roads_2000[1:]
    connected_roads_200 = connected_roads_200[1:]

    # Adding new connected roads (from non connected roads) to the pool of connected roads
    non_connected_roads_200['inter_new_geo'] = non_connected_roads_200.intersects(df_po.unary_union)
    index_to_add = non_connected_roads_200[non_connected_roads_200['inter_new_geo']==True].index

    connected_roads_200 = connected_roads_200.append(non_connected_roads_200.loc[index_to_add,:])
    non_connected_roads_200 = non_connected_roads_200.drop(index_to_add)

    connected_roads_2000 = connected_roads_2000.append(non_connected_roads_2000.loc[index_to_add,:])
    non_connected_roads_2000 = non_connected_roads_2000.drop(index_to_add)
    
    i=i+1
    logging.info('Remaining Rows %s %s'%(len(connected_roads_2000),len(non_connected_roads_200)))

In [20]:
df_200['Cost_upgrade'] = df_200['Length_Roads'] * cost_per_km
df_200['Cumu_cost'] = df_200['Cost_upgrade'].cumsum()
df_200['Cumu_Pop'] = df_200['New_People'].cumsum()
df_200['RAI'] = (df_200['Cumu_Pop']+Pop_rur_PS)/stats_rur

In [21]:
df_200.to_file(os.path.join(output_country,'%s.shp'%country ))